# Softmax

The softmax function is described in detail in [Wikipedia](https://en.wikipedia.org/wiki/Softmax_function).

In short, it returns a probability distribution for the values in a vector **z**. It is defined as, for each element $z_j$ in vector **z**:

\begin{equation}
\sigma(z)_{j} = \frac{e^{z_{j}}}{\Sigma_{k=1}^K e^{z_{k}}}
\end{equation}

Recognize that it *scales* the exponent of each element (= $e^{z_j}$) by dividing it by a constant sum, which is the summation of the exponents of all the elements of the vector (= $\Sigma_{k=1}^K e^{z_{k}}$).

If there is just one value in **z**, it translates to:

\begin{equation}
\sigma(z)_{1} = \frac{e^{z_{j}}}{\Sigma_{k=1}^1 e^{z_{k}}} = \frac{e^{z_1}}{e^{z_1}} = 1
\end{equation}

For two values:

\begin{equation}
\begin{split}
\sigma(z)_{1} &= \frac{e^{z_1}}{e^{z_1} + e^{z_2}} \\
\sigma(z)_{2} &= \frac{e^{z_2}}{e^{z_1} + e^{z_2}} \\
\end{split}
\end{equation}

And so on.

Note that the sum of the softmax result is always 1.

In [13]:
import math

def softmax(vector):
    s = 0
    for x in vector:
        s = s + math.exp(x)
    return [math.exp(x) / s for x in vector]

In [14]:
softmax([8])

[1.0]

In [15]:
softmax([3, 3])

[0.5, 0.5]

In [22]:
softmax([1, 2])

[0.2689414213699951, 0.7310585786300049]

In [16]:
v = [5, 0, 2, 3, 1, 4]
softmax(v)

[0.6336913225737218,
 0.00426977854528211,
 0.03154963320110001,
 0.08576079462509835,
 0.011606461431184656,
 0.23312200962361299]

In [17]:
sum(softmax(v))

0.9999999999999999

# Markov Decision Processes

**Markov Chain**: a process with a fixed number of _states_, where an agent _randomly_ evolves from state to state:

<img src="https://www.analyticsvidhya.com/wp-content/uploads/2014/07/transition1.png" />

**Markov Decision Process**: like a Markov Chain, except that the agent has to choose one of possible _actions_ first. The state transition probability depends on the action chosen, and some transitions may return a reward or penalty.

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/2/21/Markov_Decision_Process_example.png/400px-Markov_Decision_Process_example.png" />

The agent's _goal_ is to find a policy that will maximize the rewards over time.

**Optimal State Value** of a state $s$: the sum of all discounted future rewards the agent can expect on average after it reaches a state _s_, assuming it acts optimally.

Defined in the _recursive_ Bellman Optimality Equation, for all $s$:

\begin{equation}
V^*(s) = \max_a \sum_{s'} T(s,a,s') \big[ R(s,a,s') + \gamma \cdot V^*(s') \big]
\end{equation}

Where:

Math term   | Meaning
------------|:-------
$T(s,a,s')$ | The transition probability from state $s$ to state $s'$, giving that the agent chose action $a$.
$R(s,a,s')$ | The reward that the agent gets when it goes from state $s$ to state $s'$, given that the agent chose action $a$.
$\gamma$    | The discount rate.

So this formula looks recursively through all actions $a$ to _maximize_ the expected future rewards, simply put.

This can be found inductively, for all $s$:

\begin{equation}
\begin{split}
V_0(s)     & \leftarrow 0 \\
V_{k+1}(s) & \leftarrow \max_a \sum_{s'} T(s,a,s') \big[ R(s,a,s') + \gamma \cdot V_k(s') \big]
\end{split}
\end{equation}

Now we know the optimal state _value_ $V^*(s)$, but not which action $a$ to take. That's where the next definition comes in:

**Q-Value** of a state-action pair $(s,a)$: the sum of discounted future rewards the agent can expect on average after it reaches the state $s$ and chooses action $a$, but _before_ it sees the outcome of this action, assuming it acts optimally after that action.

Defined in the Q-Value Iteration algorithm, very similar to the Bellman Equality Equation, for all $(s,a)$:

\begin{equation}
\begin{split}
Q_0(s,a)     & \leftarrow 0 \\
Q_{k+1}(s,a) & \leftarrow \sum_{s'} T(s,a,s') \big[ R(s,a,s') + \gamma \cdot \max_{a'} Q_k(s',a') \big]
\end{split}
\end{equation}

In [17]:
import numpy as np
nan = np.nan  # represents impossible actions

# Please reference figure 16-8 in the book.

# shape = [s, a, s']
T = np.array([
    # ---- a0 -----    ---- a1 -----    ---- a2 -----
    # S'0  S'1  S'2    S'0  S'1  S'2    S'0  S'1  S'2
    [[0.7, 0.3, 0.0], [1.0, 0.0, 0.0], [0.8, 0.2, 0.0]],  # S0
    [[0.0, 1.0, 0.0], [nan, nan, nan], [0.0, 0.0, 1.0]],  # S1
    [[nan, nan, nan], [0.8, 0.1, 0.1], [nan, nan, nan]]   # S2
])

# shape = [s, a, s']
R = np.array([
    # ---- a0 -----    ---- a1 -----    ---- a2 -----
    # S'0  S'1  S'2    S'0  S'1  S'2    S'0  S'1  S'2
    [[10., 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0]],   # S0
    [[0.0, 0.0, 0.0], [nan, nan, nan], [0.0, 0.0, -50.]],  # S1
    [[nan, nan, nan], [40., 0.0, 0.0], [nan, nan, nan]]    # S2
])

possible_actions = [
    [0, 1, 2],  # S0
    [0, 2],     # S1
    [1]         # S2
]

In [18]:
Q = np.full((3, 3), -np.inf)  # -inf for impossible actions

for state, actions in enumerate(possible_actions):
    Q[state, actions] = 0.0

In [19]:
discount_rate = 0.95
n_iterations  = 100

for iteration in range(n_iterations):
    Q_prev = Q.copy()
    for s in range(3):
        for a in possible_actions[s]:
            Q[s, a] = np.sum([
                T[s, a, sp] * (R[s, a, sp] + discount_rate * np.max(Q_prev[sp]))
                for sp in range(3)
            ])

In [20]:
Q

array([[ 21.88646117,  20.79149867,  16.854807  ],
       [  1.10804034,         -inf,   1.16703135],
       [        -inf,  53.8607061 ,         -inf]])

In [21]:
np.argmax(Q, axis=1)  # optimal action for each state

array([0, 2, 1])

**Q-Learning algorithm**: adaption of the Q-Value Iteration algorithm where the transition probabilities and the rewards are initially unknown.

\begin{equation}
\begin{split}
Q_0(s,a)     & \leftarrow 0 \\
Q_{k+1}(s,a) & \leftarrow (1 - \alpha) Q_k(s,a) + \alpha \big( r + \gamma \cdot \max_{a'} Q_k(s',a') \big)
\end{split}
\end{equation}

In [29]:
import numpy.random as rnd

learning_rate0      = 0.05
learning_rate_decay = 0.1
n_iterations        = 20000

s = 0  # start in state 0

Q = np.full((3, 3), -np.inf)
for state, actions in enumerate(possible_actions):
    Q[state, actions] = 0.0

for iteration in range(n_iterations):
    a  = rnd.choice(possible_actions[s])  # choose an action (randomly)
    sp = rnd.choice(range(3), p=T[s, a])  # pick next state using T[s, a]
    reward = R[s, a, sp]

    learning_rate = learning_rate0 / (1 + iteration * learning_rate_decay)
    
    Q[s, a] = learning_rate * Q[s, a] + (1 - learning_rate) * (reward + discount_rate * np.max(Q[sp]))
    
    s = sp  # move to next state

In [30]:
Q

array([[ 146.52059407,  139.19456437,  113.28214669],
       [ 113.28146025,          -inf,  115.02178305],
       [         -inf,  179.19442723,          -inf]])

In [31]:
np.argmax(Q, axis=1)

array([0, 2, 1])